In [7]:
import pandas as pd
from fuzzywuzzy import fuzz
import useful_functions as use

In [106]:
edw_dea = pd.read_csv('../../Data/DEA/DEA_Only.csv')
aims_dea = pd.read_csv('../../Data/DEA/DEA_Only_AIMS.csv')

In [204]:
dea = pd.read_csv('../../Data/DEA/DEA_ONLY_PPMA.csv')
dea.ME = use.fix_me(dea.ME)

In [107]:
edw_dea.ME = use.fix_me(edw_dea.ME)
edw_dea['IQVIA_ME'] = [x[0:10] for x in edw_dea.ME]
aims_dea.ME = use.fix_me(aims_dea.ME)
aims_dea['IQVIA_ME'] = [x[0:10] for x in aims_dea.ME]

In [92]:
hum = pd.read_csv('../../Data/Humach/HUMACH_ALL_CONFIRMED.csv', encoding='latin')

In [93]:
hum['ME'] = use.fix_me(hum.PHYSICIAN_ME_NUMBER)

In [82]:
edw_hum = hum[hum.ME.isin(dea.ME)]
aims_hum = hum[hum.ME.isin(dea.ME)]

In [108]:
with_hum_edw = pd.merge(hum, edw_dea, on='ME')
with_hum_aims = pd.merge(hum, aims_dea, on='ME')

In [205]:
with_hum = pd.merge(hum, dea, on='ME')

In [45]:
def is_a_match(thing_1, thing_2):
    thing_1 = thing_1.upper()
    thing_2 = thing_2.upper()
    if thing_1 == thing_2:
        match = True
    elif thing_1 in thing_2:
        match = True
    elif thing_2 in thing_1:
        match = True
    elif fuzz.ratio(thing_1, thing_2)>75:
        match = True
    else:
        match = False
    return(match)

def is_match(thing_1, thing_2):
    thing_1 = str(thing_1)
    thing_2 = str(thing_2)
    if thing_1 == thing_2:
        match = True
    elif thing_1 in thing_2:
        match = True
    elif thing_2 in thing_1:
        match = True
    else:
        match = False
    return(match)

In [207]:
with_hum

,PHYSICIAN_ME_NUMBER,PHYSICIAN_FIRST_NAME,PHYSICIAN_MIDDLE_NAME,PHYSICIAN_LAST_NAME,SUFFIX_x,DEGREE,OFFICE_ADDRESS_LINE_1,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,...,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,entity_id,comm_id,src_cat_code
0,1.020015e+08,JONATHAN,C,NEWTON,NaN,NaN,STE 400 BLDG 2,1121 JOHNSON FERRY RD,MARIETTA,GA,...,C032,NEWTON,CHRISTOPHER,J,NaN,NaN,NaN,1236075,11912330,DEA
1,1.020015e+08,JONATHAN,C,NEWTON,NaN,NaN,BLD 2,1121 JOHNSON FERRY RD,MARIETTA,GA,...,C032,NEWTON,CHRISTOPHER,J,NaN,NaN,NaN,1236075,11912330,DEA
2,1.020017e+08,PETER,ALEXANDER,NIXON,NaN,NaN,STE 200,801 W 38TH ST,AUSTIN,TX,...,C022,NaN,NaN,NaN,NaN,NaN,NaN,1236231,10316099,DEA
3,1.020017e+08,PETER,ALEXANDER,NIXON,NaN,NaN,STE 200,801 W 38TH ST,AUSTIN,TX,...,C022,NaN,NaN,NaN,NaN,NaN,NaN,1236231,10316099,DEA
4,1.020017e+08,PETER,ALEXANDER,NIXON,NaN,NaN,STE 200,801 W 38TH ST,AUSTIN,TX,...,C022,NaN,NaN,NaN,NaN,NaN,NaN,1236231,10316099,DEA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30960,3.503771e+09,RICHARD,STORRS,KING,NaN,NaN,ELLIOT BEHAVIORAL HEALTH SERVICES,40 BUTTRICK RD,LONDONDERRY,NH,...,C048,NaN,NaN,NaN,NaN,NaN,NaN,7013184,6981287,DEA
30961,3.050113e+10,UMAYR,AHMAD,AZIMI,NaN,NaN,STE B,44237 MICHIGAN AVE,CANTON,MI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44080370,24626981,DEA
30962,1.175130e+09,TRISHA,NaN,PLASTINI,NaN,NaN,NaN,525 OKOCHOBEE BLVD FL 14,WEST PALM BEACH,FL,...,NaN,STERLICCHI,NaN,TRISHA,NaN,NaN,NaN,43395608,20975172,DEA
30963,4.177990e+09,RACHEL,NaN,GOLDSTEIN,NaN,NaN,NaN,850 W IRVING PARK RD,CHICAGO,IL,...,C021,NaN,NaN,NaN,NaN,NaN,NaN,9816413,10314017,DEA


In [209]:
with_hum = with_hum.fillna('None')

In [210]:
matches = []
for row in with_hum.itertuples():
    match = False 
    if row.OFFICE_ADDRESS_STATE == row.STATE:
        if is_match(row.OFFICE_ADDRESS_ZIP, row.ZIP):
            if is_a_match(row.OFFICE_ADDRESS_LINE_2, row.MAILING_LINE_1):
                match = True
    matches.append(match)

In [109]:
matches_2 = []
for row in with_hum_aims.itertuples():
    match = False 
    if row.OFFICE_ADDRESS_STATE == row.STATE:
        if is_match(row.OFFICE_ADDRESS_ZIP, row.ZIP):
            if is_a_match(row.OFFICE_ADDRESS_LINE_2, row.ADDR_1):
                match = True
    matches_2.append(match)

In [211]:
with_hum['HUMACH_VERIFIED'] = matches
# with_hum_aims['HUMACH_VERIFIED'] = matches_2

In [212]:
with_hum.WSLIVE_FILE_DT = pd.to_datetime(with_hum.WSLIVE_FILE_DT)
# with_hum_aims.WSLIVE_FILE_DT = pd.to_datetime(with_hum_aims.WSLIVE_FILE_DT)

In [87]:
with_hum.sort_values('WSLIVE_FILE_DT').drop_duplicates('ME',keep='last').to_csv('EDW_HUMACH_VERIFIED_DEA_POLOS.csv', index=False)

In [214]:
with_hum[(with_hum.HUMACH_VERIFIED==True)&with_hum.TOP_CD==20].sort_values('WSLIVE_FILE_DT').drop_duplicates('ME',keep='last')

,PHYSICIAN_ME_NUMBER,PHYSICIAN_FIRST_NAME,PHYSICIAN_MIDDLE_NAME,PHYSICIAN_LAST_NAME,SUFFIX_x,DEGREE,OFFICE_ADDRESS_LINE_1,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,...,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,entity_id,comm_id,src_cat_code,HUMACH_VERIFIED


In [115]:
with_hum_aims[with_hum_aims.HUMACH_VERIFIED==True].sort_values('WSLIVE_FILE_DT').drop_duplicates('ME',keep='last').to_csv('../../Data/DEA/AIMS_HUMACH_VERIFIED_DEA_POLOS.csv', index=False)

In [221]:
dea[dea.ME.isin(FUCK.ME)&(dea.TOP_CD==20)]

,ME,PARTY_ID,ENTITY_ID,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,...,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,entity_id,comm_id,src_cat_code
23,00102060491,3460277,27063890,A,1,1.0,ROBERT P NAFTEL MD,NAFTEL,ROBERT,PARTLOW,...,C000,NaN,NaN,NaN,NaN,NaN,NaN,27063890,18854581,DEA
97,00102760077,1845058,1267067,A,1,NaN,MARK L BARNARD MD,BARNARD,MARK,LUTHER,...,C007,NaN,NaN,NaN,NaN,NaN,NaN,1267067,13962810,DEA
113,00102801041,1845261,1274906,A,1,1.0,R S MC FARLAND III MD,MC FARLAND,R,SCOTT,...,C003,MC FARLAND,S,RICHARD,MC FARLAND,SCOTT,R,1274906,14412797,DEA
121,00102830695,1850914,1281263,A,1,2.0,MATTHEW A HUTCHINSON MD,HUTCHINSON,MATTHEW,ALAN,...,C020,NaN,NaN,NaN,NaN,NaN,NaN,1281263,11048538,DEA
145,00102910354,1845592,1297721,A,1,1.0,MARTA A CRISPENS MD,CRISPENS,MARTA,ANN,...,C000,NaN,NaN,NaN,NaN,NaN,NaN,1297721,9916280,DEA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31009,91502720380,2828664,14492409,A,1,1.0,REDA WILLIAM MD,WILLIAM,REDA,NaN,...,C770,MORCOS,W,REDA,NaN,NaN,NaN,14492409,12315468,DEA
31141,91603120065,6424439,45380851,A,1,1.0,AFRAH A ALI MD,ALI,AFRAH,ABDUL WAHID,...,C001,NaN,NaN,NaN,NaN,NaN,NaN,45380851,18378440,DEA
31144,91702900021,2775974,14534399,A,1,1.0,ANDREW G BEST MD,BEST,ANDREW,GREGORY,...,R006,NaN,NaN,NaN,NaN,NaN,NaN,14534399,7428980,DEA
31216,91905880034,2690790,14572762,A,1,1.0,TUSHAR C PATEL MD,PATEL,TUSHAR,C,...,C026,PATEL,CHANDRAKANT,T,NaN,NaN,NaN,14572762,14631679,DEA


In [180]:
eems = aims_dea[~aims_dea.IQVIA_ME.isin(tri.IQVIA_ME)&~aims_dea.ME.isin(verified_aims.ME)]

In [196]:
EEMS = pd.merge(eems, PPD[['ENTITY_ID','ME','TOP_CD']])[eeds.columns]

In [190]:
eeds.ME = use.fix_me(eeds.ME)
PPD.ME = use.fix_me(PPD.ME)

In [199]:
FUCK = pd.concat([eeds, EEMS]).drop_duplicates('ME')

In [235]:
FUCK[['ME', 'ENTITY_ID', 'PARTY_ID', 'POST_CD_ID', 'COMM_ID']].to_excel('../../Data/Unverified_DEA_Sourced_POLOs.xlsx', index=False)

In [192]:
eeds = pd.merge(PPD[['ME','ENTITY_ID','TOP_CD']], eeds, on='ME')

In [177]:
eeds = pd.read_csv('../../Data/DEA/THE_END.csv')

In [123]:
verified_aims = with_hum_aims[with_hum_aims.HUMACH_VERIFIED==True]

In [131]:
verified_edw = with_hum_edw[with_hum_edw.HUMACH_VERIFIED==True]

In [129]:
with_hum_edw[with_hum_edw.HUMACH_VERIFIED==True].sort_values('WSLIVE_FILE_DT').drop_duplicates('ME',keep='last')

,PHYSICIAN_ME_NUMBER,PHYSICIAN_FIRST_NAME,PHYSICIAN_MIDDLE_NAME,PHYSICIAN_LAST_NAME,SUFFIX,DEGREE,OFFICE_ADDRESS_LINE_1,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,...,ADDR_2,ADDR_3,CITY,STATE,ZIP,FROM_DT,COMM_ID,PURPOSE_TYPE_DESC,IQVIA_ME,HUMACH_VERIFIED
6940,5.180403e+08,DAFNA,LORGE,SCHWARTZ,NaN,MD,STE 520,1130 NW 22ND AVE,PORTLAND,OR,...,Suite 520,Women's Healthcare Associates Llc,Portland,OR,97210,2021-12-10 21:07:20.696456,28179998,Office,0051804031,True
83970,4.112050e+09,DANIEL,SCOTT,HECKMAN,NaN,MD,NaN,801 OSTRUM ST PPHP 2ND FLOOR,BETHLEHEM,PA,...,Pphp 2nd Floor,St Luke's Orthopaedic Specialists,Bethlehem,PA,18015,2021-12-10 07:28:05.989429,14757836,Office,0411205027,True
12634,1.003820e+09,WAVERLY,BURNARD,CLANTON,NaN,MD,NaN,1357 W 103RD ST,CHICAGO,IL,...,Advocate Medical Group,NaN,Chicago,IL,60643,2021-11-12 07:10:53.800785,18204796,Office,0100382024,True
93423,4.720040e+09,STEPHEN,BROCK,BLANKENSHIP,NaN,MD,DEPT OF EMERGENCY MEDICINE,16000 JOHNSTON MEMORIAL DR,ABINGDON,VA,...,Johnston Memorial Hospital,Virginia Highlands Anesthesia,Abingdon,VA,24211,2021-12-10 07:28:02.076497,15687005,Office,0472004032,True
24116,1.643910e+09,EDWARD,V,CHAN,NaN,MD,STE 501,128 MOTT ST STE 501,NEW YORK,NY,...,Suite 501,1962,New York,NY,10013,2019-07-26 21:09:16.136981,23688339,Office,0164391020,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91671,4.705850e+09,MICHAEL,GLENN,CARLSON,NaN,NaN,NaN,330 23RD AVE N STE 500,NASHVILLE,TN,...,Ste 500,NaN,Nashville,TN,37203,2021-12-10 07:27:16.007370,27856317,Office,0470585015,True
90674,4.504110e+09,NICOLE,SHARON,LINN,NaN,NaN,STE 102,131 MEDICAL PARK RD,MOORESVILLE,NC,...,Piedmont Healthcare Pa,NaN,Mooresville,NC,28117,2021-12-10 07:28:48.508239,21684015,Office,0450411025,True
90425,4.501961e+09,RICHARD,TAYLOR,WILLIAMS,NaN,NaN,STE 300,2728 SUNSET BLVD,WEST COLUMBIA,SC,...,Ste 300,Limited To Official Lexington Medical Ce,West Columbia,SC,29169,2021-12-10 07:27:15.097845,24635543,Office,0450196059,True
56786,3.306051e+09,STACEY,BETH,TROOSKIN,NaN,NaN,NaN,1233 LOCUST ST FL 5,PHILADELPHIA,PA,...,NaN,NaN,Philadelphia,PA,19107,2021-10-12 20:51:39.498545,19762470,Office,0330605099,True


In [175]:
edw_dea[~edw_dea.ME.isin(tri_edw.ME)&~edw_dea.ME.isin(verified_edw.ME)].to_csv('../../Data/DEA/ONLY_DEA_EDW.csv', index=False)

In [178]:
PPD = pd.read_csv('../../Data/PPD/ppd_analysis_file.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17,18,24,25,57,64,65,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [140]:
(2631/119267)*147302

3249.4450434738865

In [144]:
(2631/90917)*84512

2445.6490205352134

In [163]:
test = edw_dea[~edw_dea.ME.isin(verified_edw.ME)]

In [148]:
dpcs = list(ppd[ppd.TOP_CD==20].ME)

In [151]:
nulls = pd.read_csv('../../Data/DEA/null_NOT_DEAs.csv')

In [152]:
nulls.columns

Index(['entity_id', 'comm_id', 'src_cat_code', 'comm_type', 'comm_usage'], dtype='object')

In [168]:
tri_edw = pd.read_csv('../../Data/DEA/EDW_TRIANGULATION.csv')

In [171]:
tri_edw.ME = use.fix_me(tri_edw.ME)

In [166]:
test.to_csv('../../Data/DEA/DEA_NOT.csv', index=False)

In [164]:
pd.merge(test, nulls, on=['entity_id','comm_id'], suffixes = ['','_other']).drop_duplicates('ME')

KeyError: 'entity_id'

In [158]:
YEA[YEA.ME.isin(dpcs)]

,PARTY_ID,ME,POST_CD_ID,CAT_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE,ZIP,...,PURPOSE_TYPE_DESC,ENTITY_ID_x,entity_id,comm_id,src_cat_code,ENTITY_ID_y,IQVIA_ME,src_cat_code_other,comm_type,comm_usage
0,1889093,00102920414,16386128,NaN,4327 Kennesaw Dr,NaN,NaN,Mountain Brk,AL,35213,...,Office,1299918,1299918,6388882,DEA,1299918,0010292041,ACS,HM,PP
2,1893670,00301960298,16475471,NaN,77 W Forest Ave,Ste 201,NaN,Flagstaff,AZ,86001,...,Office,1369221,1369221,8254921,DEA,1369221,0030196029,MBSHP-MAIL,HM,PP
7,1906782,00301970749,16956195,NaN,29 Bretton Rd,NaN,NaN,West Hartford,CT,6119,...,Office,1371015,1371015,7193710,DEA,1371015,0030197074,ACS,HM,PP
11,1894918,00401940171,26712693,NaN,123 Bluefish Ave,NaN,NaN,Russellville,AR,72802,...,Office,1451589,1451589,20753214,DEA,1451589,0040194017,ACXIOMNCOA,HM,PP
13,1890942,00502740113,9837908,NaN,2977 Rustle Ct,NaN,NaN,Fairfield,CA,94534,...,Office,1516251,1516251,7436030,DEA,1516251,0050274011,ACS,HM,PP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1894,6278003,02275090878,18632904,NaN,915 Union St Ste 4,Emmc Walk-In Care,NaN,Bangor,ME,4401,...,Office,45036754,45036754,10987001,DEA,45036754,0227509087,NPI,OF,PO
1896,5656750,42201122087,23612720,NaN,204 Patrick Ave,NaN,NaN,Urbana,OH,43078,...,Office,43259810,43259810,19120549,DEA,43259810,4220112208,NPI,OF,PP
1897,3582275,70402040619,26696655,NaN,100 Evergreen Sq SW,NaN,NaN,Pine City,MN,55063,...,Office,28650280,28650280,20740798,DEA,28650280,7040204061,NPI,OF,PP
1899,6828001,04177111665,10604160,NaN,5 Palisades Dr,Ste 200,NaN,Albany,NY,12205,...,Office,46576877,46576877,6729356,DEA,46576877,0417711166,LICENSURE,OF,PP


In [215]:
dea

,ME,PARTY_ID,ENTITY_ID,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,...,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,entity_id,comm_id,src_cat_code
0,00102000358,1891435,1235022,A,1,1.0,ROBERT N AXON MD,AXON,ROBERT,NEAL,...,C006,NaN,NaN,NaN,NaN,NaN,NaN,1235022,9859774,DEA
1,00102001486,1891444,1236075,A,1,1.0,JONATHAN C NEWTON MD,NEWTON,JONATHAN,C,...,C032,NEWTON,CHRISTOPHER,J,NaN,NaN,NaN,1236075,11912330,DEA
2,00102001729,1903182,1236231,A,1,1.0,PETER A NIXON MD,NIXON,PETER,ALEXANDER,...,C022,NaN,NaN,NaN,NaN,NaN,NaN,1236231,10316099,DEA
3,00102010825,1843292,1237778,A,1,1.0,FLORANCE HYPPOLITE MD,HYPPOLITE,FLORANCE,NaN,...,C004,NaN,NaN,NaN,NaN,NaN,NaN,1237778,9608258,DEA
4,00102010868,1891459,1237830,A,1,1.0,BRENT H LIMBAUGH MD,LIMBAUGH,BRENT,HAMMAL,...,C097,NaN,NaN,NaN,NaN,NaN,NaN,1237830,11361455,DEA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31303,95708880117,3322299,21050714,A,1,1.0,ADNAN BEGOVIC MD,BEGOVIC,ADNAN,NaN,...,C024,NaN,NaN,NaN,NaN,NaN,NaN,21050714,8476683,DEA
31304,95709080017,5847309,43802235,A,1,1.0,BLERIM ARIFI MD,ARIFI,BLERIM,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43802235,14007119,DEA
31305,95709890035,3267143,20333686,A,1,2.0,ENISA GOLJO MD,GOLJO,ENISA,NaN,...,C073,NaN,NaN,NaN,NaN,NaN,NaN,20333686,10503557,DEA
31306,95712000024,3683183,29362706,A,1,1.0,DANIJELA LEVACIC MD,LEVACIC,DANIJELA,NaN,...,C036,NaN,NaN,NaN,NaN,NaN,NaN,29362706,22170513,DEA


In [ ]:
pd.read_csv('../../Data/')